## GEOCODING- Add Block Group to NFIRS Data
### 1/21/20
Add Census block group information to NFIRS data using Census geocode API

In [1]:
import pandas as pd
import pathlib

### Set filepaths

In [2]:
nfirs_path = 'NFIRS_2009_2016_Combined_Census_Tract.csv'

### Prep sub directories

In [3]:
pathlib.Path("./data_scratch").mkdir(parents=True, exist_ok=True)
pathlib.Path("./data_scratch/input").mkdir(parents=True, exist_ok=True)
pathlib.Path("./data_scratch/output").mkdir(parents=True, exist_ok=True)

### Prep existing NFIRS file for Census API

In [4]:
# load nfirs data for batch processing
cols_to_use = ['ID', 'state', 'street', 'city', 'zip5']
col_dtypes = {x:str for x in cols_to_use}

nfirs_lookup = pd.read_csv(nfirs_path, dtype = col_dtypes, usecols = cols_to_use, encoding='latin-1')

# initial data cleaning
rn = {'ID':'Unique ID', 'state':'State', 'street':'Street Address', 'city':'City', 'zip5':'Zip'}
nfirs_lookup.rename(columns=rn, inplace=True)
nfirs_lookup = nfirs_lookup[['Unique ID', 'Street Address', 'City', 'State', 'Zip']].copy()
print(len(nfirs_lookup.index), 'rows')
nfirs_lookup.head()

1959106 rows


,Unique ID,Street Address,City,State,Zip
0,rcp2_id1,1700 ANGUS WAY,JUNEAU,AK,99801
1,rcp2_id2,3221 PIONEER AVE,JUNEAU,AK,99801
2,rcp2_id3,10231 HERON WAY,JUNEAU,AK,99801
3,rcp2_id4,6590 GLACIER HWY,JUNEAU,AK,99801
4,rcp2_id5,6590 GLACIER HWY,JUNEAU,AK,99801


In [5]:
# split into blocks of 10K (max batch size for Census API)
nrow = len(nfirs_lookup.index)
file_num = 1
cur_row = 0
while cur_row <= nrow -1:
    temp = nfirs_lookup.loc[cur_row:cur_row + 9999, :].copy()
    filename = 'data_scratch//input//nfirs_part{0}.csv'.format(file_num)
    temp.to_csv(filename, index=False, header=False)
    file_num += 1
    cur_row += 10000
max_file = file_num - 1
print('files 1 to {0} created'.format(max_file))

files 1 to 196 created


### Geocode addresses through Census API
Information on the API found here: https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
The API accepts at most 10,000 lines in the block input.  Each file takes approximately 15-25 minutes to run.  It took approximately 48 hours of machine time to process all of the groups.  A few of the files did not process correctly -- caught by the code in the following cell -- and re-run manually.

The following API parameters were used:  
* *benchmark*=Public_AR_Census2010  
* *vintage*=Census2010_Census2010

Sample curl command:
```
curl --form addressFile=@nfirs_part1.csv --form benchmark=Public_AR_Census2010 --form vintage=Census2010_Census2010 https://geocoding.geo.census.gov/geocoder/geographies/addressbatch --output api_output1.csv
```

Sample Bash program:
```
#!/bin/bash

for i in {1..196}
do
   lf="@data_scratch//input//nfirs_part${i}.csv"
   of="api_output${i}.csv"
   curl --form addressFile=$lf --form benchmark=Public_AR_Census2010 --form vintage=Census2010_Census2010  https://geocoding.geo.census.gov/geocoder/geographies/addressbatch --output $of
   mv $of data_scratch/output/
   wc -l data_scratch/output/$of
done
```

This was run for all files 1-```max_file```, creating data_scratch/output/api_output#.csv files.

### Process Data from API

#### Build RCP2 id to full geoid lookup

In [11]:
# process and append each file
geoid_lookup = pd.DataFrame(columns=['ID', 'geoid'])
cols = ['ID', 'address', 'match_status', 'match_type', 'matched_address', 'lat_long', 'skip1', 'skip2', 'state', 'county', 'tract', 'block']
col_dtypes = {x:str for x in cols}

for file_num in range(1, 197):
    try:
        filename = 'data_scratch//output//api_output{0}.csv'.format(file_num)
        temp = pd.read_csv(filename, names=cols, dtype=col_dtypes)
        temp['geoid'] = temp['state'] + temp['county'] + temp['tract'] + temp['block']
        temp = temp[['ID', 'geoid']].copy()
        geoid_lookup = geoid_lookup.append(temp)
    except Exception as ex:
        print('issue with file {0}: {1}'.format(file_num, ex))
        
geoid_lookup['sort_num'] = geoid_lookup['ID'].str.replace('^rcp2_id', '').astype(int)
geoid_lookup.sort_values('sort_num', inplace=True)
geoid_lookup.reset_index()
geoid_lookup = geoid_lookup.drop('sort_num', axis='columns')
print('{0} lines loaded'.format(len(geoid_lookup.index)))
geoid_lookup.head()

1959106 lines loaded


,ID,geoid
9752,rcp2_id1,021100005001018
9751,rcp2_id2,021100006002006
9757,rcp2_id3,021100001003030
9756,rcp2_id4,021100004001003
9755,rcp2_id5,021100004001003


In [12]:
# additional cleaning
geoid_lookup['geoid_tract'] = geoid_lookup['geoid'].str[0:11]
geoid_lookup['geoid_blockgroup'] = geoid_lookup['geoid'].str[0:12]
geoid_lookup.rename(columns={'geoid': 'geoid_full', 'ID': 'id'}, inplace=True)

geoid_lookup.head()

,id,geoid_full,geoid_tract,geoid_blockgroup
9752,rcp2_id1,021100005001018,02110000500,021100005001
9751,rcp2_id2,021100006002006,02110000600,021100006002
9757,rcp2_id3,021100001003030,02110000100,021100001003
9756,rcp2_id4,021100004001003,02110000400,021100004001
9755,rcp2_id5,021100004001003,02110000400,021100004001


In [13]:
# export geoid lookup
geoid_lookup.to_csv('..//Data//processed//geoid_lookup.csv', index=False)

#### Reload NFIRS file for QC

In [14]:
# load nfirs data
cols_to_use = ['ID', 'GEOID', 'state']
col_dtypes = {x:str for x in cols_to_use}

nfirs = pd.read_csv(nfirs_path, dtype = col_dtypes, usecols = cols_to_use, encoding='latin-1')

# initial data cleaning
rn = {x: x.lower() for x in cols_to_use if x!=x.lower()}
nfirs.rename(columns=rn, inplace=True)
nfirs['geoid'] = (nfirs['geoid'].str[:-2].str.zfill(11))

print('{0} ids without geoid initially'.format(sum(nfirs['geoid'].isna())))
nfirs.head()

320195 ids without geoid initially


,id,state,geoid
0,rcp2_id1,AK,02110000500
1,rcp2_id2,AK,02110000600
2,rcp2_id3,AK,02110000100
3,rcp2_id4,AK,02110000400
4,rcp2_id5,AK,02110000400


In [26]:
comb = pd.merge(nfirs, geoid_lookup, how='left', on='id')
comb['match'] = 'Mis-match'
comb.loc[comb['geoid'] == comb['geoid_tract'], 'match'] = 'Match'
comb.loc[comb['geoid_tract'].isna(), 'match'] = 'New NA'
comb.loc[comb['geoid'].isna(), 'match'] = 'Orig NA'

print(comb['match'].value_counts())
comb['match'].value_counts(normalize=True)

Match        1497750
Orig NA       320195
New NA        131261
Mis-match       9900
Name: match, dtype: int64


Match        0.764507
Orig NA      0.163439
New NA       0.067000
Mis-match    0.005053
Name: match, dtype: float64

In [36]:
# check state and county for mis-matches
comb['old_state_county'] = comb['geoid'].str[0:5]
comb['new_state_county'] = comb['geoid_tract'].str[0:5]
comb['check_mismatch'] = 'ignore'
comb.loc[comb['match']=='Mis-match', 'check_mismatch'] = 'Mis-match'
comb.loc[(comb['match']=='Mis-match') 
                & (comb['old_state_county']==comb['new_state_county']), 'check_mismatch'] = 'Match'

temp = comb.loc[comb['match'] == 'Mis-match', ['id', 'check_mismatch']].copy()
print(temp['check_mismatch'].value_counts())
temp['check_mismatch'].value_counts(normalize=True)

Match        9281
Mis-match     619
Name: check_mismatch, dtype: int64


Match        0.937475
Mis-match    0.062525
Name: check_mismatch, dtype: float64

**Notes:**  
It's unclear why there are new NA's and mis-matches.  One possibility is that the new NA's are the result of additional address cleaning from the first round that is not reflected in the latest NFIRS file.  This process used the separate, original, address fields rather than the matched addresses.  
  
The mis-matches could be due to different *benchmark* and *vintage* parameters for the Census API between the first round of geocoding and this version.  93% of the mis-matches line up at the state & county level.

#### Add match status to geoid_lookup

In [48]:
new_lookup = comb.loc[:, ['id', 'geoid_full', 'geoid_tract', 'geoid_blockgroup', 'match', 'check_mismatch']].copy()
new_lookup['match_status'] = new_lookup['match']
new_lookup.loc[new_lookup['check_mismatch']=='Match', 'match_status'] = 'Mis-match: state/county same'
new_lookup.loc[new_lookup['check_mismatch']=='Mis-match', 'match_status'] = 'Mis-match: state/county different'
new_lookup.drop(['match', 'check_mismatch'], axis='columns', inplace=True)

# update sort
new_lookup['sort_num'] = new_lookup['id'].str.replace('^rcp2_id', '').astype(int)
new_lookup.sort_values('sort_num', inplace=True)
new_lookup.reset_index()
new_lookup = new_lookup.drop('sort_num', axis='columns')

new_lookup['match_status'].value_counts()

Match                                1497750
Orig NA                               320195
New NA                                131261
Mis-match: state/county same            9281
Mis-match: state/county different        619
Name: match_status, dtype: int64

In [52]:
# export to csv
new_lookup.to_csv('..//Data//processed//geoid_lookup.csv', index=False)

**Note:**  
Lookup file is too big for Github.  
Manually uploaded to RCP2/02_Data/Master Data/NFIRS - National Fire Incident Reporting System/ folder on Google Drive.